# 0 - QB - Crossvals

## Importation des modules

In [ ]:
# Importation des modules
# Modules de base
import os
import pandas as pd
# Sklearn


## Importation des données

In [ ]:
# Importation des données de test